In [ ]:
def retrodict(state, prior_state, transition_model):
    F = transition_model.matrix()
    
    x_ll = prior_state.state_vector()
    P_ll = prior_state.covar()
    
    x_l1k = state.state_vector()
    P_l1k = state.covar()
    
    x_l1l = F @ x_ll
    P_l1l = F @ P_ll @ F.T + transition_model.covar()
    
    W_l1l = P_ll @ F.T @ P_l1l.
